# 特斯拉ptt輿情分析

## 動機和分析目的
利用ptt對特斯拉的文章，進行以下的分析，可能的分析方向如下：
+ 社群媒體分析：探索特斯拉在不同社群媒體平台上的評論和討論。如特斯拉的技術創新、環保意識等。
+ 市場趨勢預測：分析特斯拉的新產品發布、重大事件或競爭對手的動態對市場的影響，並進行股市趨勢預測。
+ 產品品質分析：使分析消費者的投訴、讚美和建議，並進行情緒分類和主題分析，探討產品品質的缺點與優點。
+ 競爭分析：分析特斯拉與其他競爭對手的比較。特斯拉和其他汽車品牌的情緒和評價，了解特斯拉市場上的定位和競爭策略。

In [38]:
import time 
import pandas as pd #pandas: 一個資料分析的函式庫，提供了DataFrame等資料格式，與資料處理的函數。
import numpy as np #numpy: 為科學運算所提供的套件。提供許多數學公式的實作以及高效率的陣列、矩陣運算。
import re #re: 正規表達式套件。
import math #math: 數學運算工具。
import datetime
from pprint import pprint
from collections import defaultdict
import multiprocessing
import os
from getpass import getpass
import seaborn as sns
from matplotlib import font_manager
import matplotlib.pyplot as plt  #matplotlib: 繪圖工具。
import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm
import jieba #jieba: 中文斷詞套件。
import jieba.analyse

import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.gensim_models
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.matutils import corpus2csc, corpus2dense, Sparse2Corpus
from transformers import BertTokenizerFast, AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification, pipeline
from sentence_transformers import SentenceTransformer
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker 
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from wordcloud import WordCloud 
from nltk import ngrams, FreqDist #NLTK: 全名為Natural Language Tool Kit，自然語言處理工具。
from collections import Counter, namedtuple #collections: 提供額外的數據類型。
import networkx as nx #networkx: 網絡圖建構、繪製工具。
from snownlp import SnowNLP #SnowNLP 針對中文設計，可以提取句子的情緒分類
# from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec, KeyedVectors
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer #CountVectorizer, TfidfTransformer: sklearn中計算詞頻與tf-idf的套件。
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict, KFold
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    precision_recall_curve,
    RocCurveDisplay
)
import guidedlda

## 資料集
資料來源：
+ 資料來源：tarflow PPT車版
+ 關鍵字：特斯拉、Tesla
+ 停用字：馬司克、Musk 
+ 資料筆數：1585篇
+ 訓練資料日期：2023/1/1 - 2024/04/30

## 情緒分析

## 標注文章
三種方式 : guilda、bert_Topic、prompt engineering

###  1. Guild LDA

In [39]:
plt.rcParams['font.sans-serif'] = ['Arial Unicode Ms']
plt.rcParams['axes.unicode_minus']=False

In [40]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [41]:
df = pd.read_csv('data/ppt_Tesla.csv', encoding = 'UTF-8')
df.dropna(subset = ['artContent'], axis=0, how='any', inplace=True)
df = df[['artDate','artTitle','artContent']]
df['artDate'] = pd.to_datetime(df['artDate'])
df.head(3)

,artDate,artTitle,artContent
0,2023-01-01 10:48:19,Re:[情報]慟！特斯拉二手價跌好多,八卦是官網特斯拉二手車\n\n比很多第三方平台都便宜\nhttps://www.tesla....
1,2023-01-01 22:42:25,[問題]特斯拉久了會掉電嗎？,各位前輩好\n 想買中古特斯拉\n里程約5萬上下的\n\n擔心電池久了會掉電\n越充越少電 ...
2,2023-01-02 12:47:31,[新聞]開特斯拉「滑手機」被勸Toyz嗆網友有毛,原文連結：\nhttps://star.setn.com/news/1232578?utm_...


In [42]:
car = df.copy()
car["artContent"] = car["artContent"].str.replace("(http|https)://.*", "", regex=True)
car["artTitle"] = car["artTitle"].str.replace("(http|https)://.*", "", regex=True)
car["artContent"] = car["artContent"].str.replace("[^\u4e00-\u9fa5]+", "", regex=True)
car["artTitle"] = car["artTitle"].str.replace("[^\u4e00-\u9fa5]+", "", regex=True)
car['content'] = car['artContent']
car['topic_label']= ""
car

,artDate,artTitle,artContent,content,topic_label
0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,
1,2023-01-01 22:42:25,問題特斯拉久了會掉電嗎,各位前輩好想買中古特斯拉里程約萬上下的擔心電池久了會掉電越充越少電里程數越少開了年後車子變成...,各位前輩好想買中古特斯拉里程約萬上下的擔心電池久了會掉電越充越少電里程數越少開了年後車子變成...,
2,2023-01-02 12:47:31,新聞開特斯拉滑手機被勸嗆網友有毛,原文連結原文內容記者王家珩徐兆緯台北報導前電競世界冠軍日前開直播雖然用自動駕駛但雙手卻完全離...,原文連結原文內容記者王家珩徐兆緯台北報導前電競世界冠軍日前開直播雖然用自動駕駛但雙手卻完全離...,
3,2023-01-02 18:01:04,討論是不是真的有些四輪覺得自己是大爺,如題本身開特斯拉最近出門覺得找車位很煩都騎家裡超舊機車基本上在路上有機車道騎機車道沒機車道就...,如題本身開特斯拉最近出門覺得找車位很煩都騎家裡超舊機車基本上在路上有機車道騎機車道沒機車道就...,
4,2023-01-02 19:21:24,新聞美國汽車召回排名揭曉第一名超過,原文連結原文內容美國汽車召回排名揭曉第一名超過萬輛車進廠處理結束迎來許多年的數據也立刻揭曉來...,原文連結原文內容美國汽車召回排名揭曉第一名超過萬輛車進廠處理結束迎來許多年的數據也立刻揭曉來...,
...,...,...,...,...,...
1581,2024-04-30 10:33:35,新聞特斯拉全新入門電動車準備量產,哪有到倍那種是油車電動車現在沒有貨物稅以美國現在價格簡單計算台灣萬只比美國售價直接換算貴一點...,哪有到倍那種是油車電動車現在沒有貨物稅以美國現在價格簡單計算台灣萬只比美國售價直接換算貴一點...,
1582,2024-04-30 12:24:31,分享撞緩撞車啦,要系統性的改善緩撞車被撞的問題建議交通部學習美國跟特斯拉的案件作法以後強制要求各車廠開啟後系...,要系統性的改善緩撞車被撞的問題建議交通部學習美國跟特斯拉的案件作法以後強制要求各車廠開啟後系...,
1583,2024-04-30 17:45:19,新聞銷售降溫賓士上季獲利銳減逾三成福斯,都轉單到特斯拉了誰跟你買雙不針對本板及有多位朋友同想法都了還在特斯拉搶走雙客群就我看賓士的衰...,都轉單到特斯拉了誰跟你買雙不針對本板及有多位朋友同想法都了還在特斯拉搶走雙客群就我看賓士的衰...,
1584,2024-04-30 19:19:58,問題,最近物色下一台車原本抓左右上週去看了特斯拉這周去看了爆預算爆很大目前用車都是十年以上換車里程...,最近物色下一台車原本抓左右上週去看了特斯拉這周去看了爆預算爆很大目前用車都是十年以上換車里程...,


In [43]:
# 設定繁體中文詞庫
jieba.set_dictionary("./dict/dict.txt.big")

# 新增stopwords
# jieba.analyse.set_stop_words('./dict/stopwords.txt') #jieba.analyse.extract_tags才會作用
with open("./dict/stopwords.txt", encoding="utf-8") as f:
    stopWords = [line.strip() for line in f.readlines()]

# 設定斷詞 function
def getToken(row):
    seg_list = jieba.cut(row, cut_all=False)
    seg_list = [
        w for w in seg_list if w not in stopWords and len(w) > 1
    ]  # 篩選掉停用字與字元數大於1的詞彙
    return seg_list

car["words"] = car["content"].apply(getToken)
car.head()

Building prefix dict from /Users/arron/Desktop/SMA_project/SMA_project/dict/dict.txt.big ...
2024-06-05 23:27:53,423 : DEBUG : Building prefix dict from /Users/arron/Desktop/SMA_project/SMA_project/dict/dict.txt.big ...
Loading model from cache /var/folders/s0/vxcd6261437fdmx2j76q9tj00000gn/T/jieba.u160d5499c10b2133fd820e5821d8ed10.cache
2024-06-05 23:27:53,424 : DEBUG : Loading model from cache /var/folders/s0/vxcd6261437fdmx2j76q9tj00000gn/T/jieba.u160d5499c10b2133fd820e5821d8ed10.cache
Loading model cost 0.517 seconds.
2024-06-05 23:27:53,941 : DEBUG : Loading model cost 0.517 seconds.
Prefix dict has been built successfully.
2024-06-05 23:27:53,941 : DEBUG : Prefix dict has been built successfully.


,artDate,artTitle,artContent,content,topic_label,words
0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,,"[八卦, 官網, 特斯拉, 二手車, 第三方, 平台, 便宜, 黨會, 勝利]"
1,2023-01-01 22:42:25,問題特斯拉久了會掉電嗎,各位前輩好想買中古特斯拉里程約萬上下的擔心電池久了會掉電越充越少電里程數越少開了年後車子變成...,各位前輩好想買中古特斯拉里程約萬上下的擔心電池久了會掉電越充越少電里程數越少開了年後車子變成...,,"[前輩, 中古, 特斯拉, 里程, 擔心, 電池, 掉電, 越充, 越少電, 里程, 數越,..."
2,2023-01-02 12:47:31,新聞開特斯拉滑手機被勸嗆網友有毛,原文連結原文內容記者王家珩徐兆緯台北報導前電競世界冠軍日前開直播雖然用自動駕駛但雙手卻完全離...,原文連結原文內容記者王家珩徐兆緯台北報導前電競世界冠軍日前開直播雖然用自動駕駛但雙手卻完全離...,,"[原文, 連結, 原文, 內容, 王家, 徐兆緯, 台北, 前電, 世界冠軍, 日前, 直播..."
3,2023-01-02 18:01:04,討論是不是真的有些四輪覺得自己是大爺,如題本身開特斯拉最近出門覺得找車位很煩都騎家裡超舊機車基本上在路上有機車道騎機車道沒機車道就...,如題本身開特斯拉最近出門覺得找車位很煩都騎家裡超舊機車基本上在路上有機車道騎機車道沒機車道就...,,"[特斯拉, 最近, 出門, 車位, 很煩, 騎家裡, 超舊, 機車, 路上, 有機, 車道,..."
4,2023-01-02 19:21:24,新聞美國汽車召回排名揭曉第一名超過,原文連結原文內容美國汽車召回排名揭曉第一名超過萬輛車進廠處理結束迎來許多年的數據也立刻揭曉來...,原文連結原文內容美國汽車召回排名揭曉第一名超過萬輛車進廠處理結束迎來許多年的數據也立刻揭曉來...,,"[原文, 連結, 原文, 內容, 美國, 汽車, 召回, 排名, 揭曉, 第一名, 超過, ..."


In [44]:
docs = car['words'].to_list()
docs[0]

['八卦', '官網', '特斯拉', '二手車', '第三方', '平台', '便宜', '黨會', '勝利']

In [45]:
dictionary = Dictionary(docs)
# Filter out words that occur less than 5 documents, or more than 99% of the documents.
dictionary.filter_extremes(no_below=5, no_above=0.99)
print(dictionary)

2024-06-05 23:27:59,750 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-06-05 23:27:59,843 : INFO : built Dictionary<28609 unique tokens: ['二手車', '便宜', '八卦', '勝利', '官網']...> from 1585 documents (total 192653 corpus positions)
2024-06-05 23:27:59,844 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<28609 unique tokens: ['二手車', '便宜', '八卦', '勝利', '官網']...> from 1585 documents (total 192653 corpus positions)", 'datetime': '2024-06-05T23:27:59.844041', 'gensim': '4.3.2', 'python': '3.11.5 (main, Sep 11 2023, 08:31:25) [Clang 14.0.6 ]', 'platform': 'macOS-14.5-arm64-arm-64bit', 'event': 'created'}
2024-06-05 23:27:59,851 : INFO : discarding 23876 tokens: [('黨會', 1), ('寵物', 4), ('少開', 1), ('掉電', 4), ('數越', 1), ('有載', 2), ('沒人收', 1), ('越充', 1), ('越少電', 1), ('一拳', 2)]...
2024-06-05 23:27:59,852 : INFO : keeping 4733 tokens which were in no less than 5 and no more than 1569 (=99.0%) documents
2024-06-05 23:27:59,856 : INFO : resulting dictionary: Dictionary<4733 

Dictionary<4733 unique tokens: ['二手車', '便宜', '八卦', '勝利', '官網']...>


In [46]:
for idx, (k, v) in enumerate(dictionary.token2id.items()):
    print(f"{k}: {v}")
    if idx > 10:
        break

二手車: 0
便宜: 1
八卦: 2
勝利: 3
官網: 4
平台: 5
特斯拉: 6
第三方: 7
一點: 8
不到: 9
中古: 10
之後: 11


In [47]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
type(corpus)

list

In [48]:
vocab = tuple(dictionary.token2id.keys())
vocab[:10]

('二手車', '便宜', '八卦', '勝利', '官網', '平台', '特斯拉', '第三方', '一點', '不到')

In [49]:
word2id = dictionary.token2id

In [50]:
seed_topic_list = [
    ["服務", "員工", "品質", "後勤", "更換", "速度","態度"],
    ["設備", "交付", "延遲", "據點","原廠"],
    ["價格","優惠","打折","折價"],
    ["安全", "過熱", "火車", "爆炸", "失控","召回","系統","自駕","煞車"],
    ["體驗", "駕駛", "操控", "公差","里程","續航力","加速","噪音","焦慮"],
]

In [51]:
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

In [52]:
# guidedlda 需要 DTM 格式作為 input，因此這邊利用 corpus2dense() 方法進行轉換
X = corpus2dense(corpus, len(dictionary), len(corpus)).T.astype(np.int64)

In [53]:
model = guidedlda.GuidedLDA(n_topics=6, n_iter=100, random_state=7, refresh=20)
model.fit(X, seed_topics=seed_topics, seed_confidence=1)

2024-06-05 23:27:59,975 : INFO : n_documents: 1585
2024-06-05 23:27:59,975 : INFO : vocab_size: 4733
2024-06-05 23:27:59,975 : INFO : n_words: 151704
2024-06-05 23:27:59,975 : INFO : n_topics: 6
2024-06-05 23:27:59,976 : INFO : n_iter: 100
2024-06-05 23:27:59,979 : WARNING : all zero row in document-term matrix found
2024-06-05 23:28:00,463 : INFO : <0> log likelihood: -1543560
2024-06-05 23:28:00,595 : INFO : <20> log likelihood: -1211838
2024-06-05 23:28:00,723 : INFO : <40> log likelihood: -1193781
2024-06-05 23:28:00,850 : INFO : <60> log likelihood: -1185532
2024-06-05 23:28:00,976 : INFO : <80> log likelihood: -1179899
2024-06-05 23:28:01,100 : INFO : <99> log likelihood: -1176149


In [54]:
# 整理／顯示主題模型結果
n_top_words = 20
topic_word = model.topic_word_

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][: -(n_top_words + 1) : -1]
    print("Topic {}: {}".format(i, " ".join(topic_words)))
    

doc_topic = model.doc_topic_
term_freq = tuple(dictionary.cfs.values())
doc_len = [sum(v for k, v in doc) for doc in corpus]

## LDAvis
pyLDAvis.enable_notebook()
p = pyLDAvis.prepare(topic_word, doc_topic, doc_len, vocab = vocab, term_frequency = term_freq)
p

Topic 0: 電動車 汽車 電池 市場 公司 特斯拉 美國 中國 生產 技術 豐田 歐洲 全球 製造商 萬輛 成本 車廠 充電 原文 電動
Topic 1: 充電 特斯拉 電車 電池 油車 車主 現在 使用 問題 目前 超充 電動車 里程 需要 車子 時間 知道 台灣 保養 價格
Topic 2: 中國 汽車 品牌 台灣 時間 銷量 報名 比亞迪 日本 市場 車馬費 台北市 網頁 活動 購買 特斯拉 大陸 車款 目前 購車
Topic 3: 特斯拉 駕駛 車主 車輛 原文 自動 發生 安全 輔助 系統 說明 心得 內容 連結 功能 電動車 召回 事故 車體 使用
Topic 4: 特斯拉 感覺 空間 駕駛 看到 知道 一下 問題 開車 直接 車子 功能 後座 方向盤 一台 業務 煞車 喜歡 加速 之後
Topic 5: 特斯拉 原文 台灣 萬元 降價 車型 銷售 品牌 心得 電動車 連結 價格 市場 售價 新車 車款 說明 台幣 交車 部分


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.235386 -0.171644       1        1  22.740121
1      0.100177 -0.117496       2        1  22.407071
0     -0.084116 -0.066538       3        1  19.631925
3      0.119626  0.281728       4        1  13.881187
5     -0.070245  0.152311       5        1  12.707101
2     -0.300828 -0.078361       6        1   8.632596, topic_info=     Term         Freq        Total Category  logprob  loglift
373    中國  1161.000000  1161.000000  Default  30.0000  30.0000
440    充電  1399.000000  1399.000000  Default  29.0000  29.0000
28    電動車  2136.000000  2136.000000  Default  28.0000  28.0000
109    駕駛   977.000000   977.000000  Default  27.0000  27.0000
149    汽車  1126.000000  1126.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
2162   符合    69.805653   132.753976   Topic6  -5.2343   1.8068
561    目前    87.753114   656.782267   Topic6  -5.0055   0.4368
6     特斯拉    94.732682  3049.129628   Topic6  -4.9290  -1.0219
436    使用    72.796896   550.001677   Topic6  -5.1924   0.4274
819    德國    67.811490   174.701539   Topic6  -5.2633   1.5033

[433 rows x 6 columns], token_table=      Topic      Freq Term
term                      
30        1  0.571208   一下
30        2  0.409836   一下
30        6  0.020492   一下
702       1  0.457421   一台
702       2  0.457421   一台
...     ...       ...  ...
109       2  0.002047   駕駛
109       4  0.749130   駕駛
2680      1  1.000337  駕駛座
2484      1  0.909333   體驗
2484      3  0.091768   體驗

[693 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 4, 6, 3])

In [55]:
topics_doc = model.transform(X)
print(len(doc_topic))

2024-06-05 23:28:02,130 : WARNING : all zero row in document-term matrix found


1585


In [56]:
for i in range(len(doc_topic)):
    print("topic :{} Document: {}".format(doc_topic[i].argmax(), ', '.join(np.array(vocab)[list(reversed(X[i,:].argsort()))[0:5]])))
    car['topic_label'][i] = doc_topic[i].argmax()

topic :5 Document: 二手車, 便宜, 八卦, 勝利, 官網
topic :1 Document: 中古, 里程, 建議, 電動車, 即可
topic :3 Document: 開車, 直播, 方向盤, 危險, 自動
topic :4 Document: 機車, 最近, 車道, 車位, 遇到
topic :3 Document: 召回, 萬輛, 美國, 福特, 揭曉
topic :0 Document: 按摩, 單一, 備註, 僅有, 優勢
topic :3 Document: 需要, 回廠, 召回, 台車, 特斯拉
topic :5 Document: 萬輛, 特斯拉, 交付, 生產, 第四季
topic :5 Document: 銷售, 進口, 市場, 車款, 變動
topic :3 Document: 特斯拉, 一名, 公路, 加州, 事故
topic :2 Document: 汽車, 一汽, 豐田, 本田, 上汽
topic :5 Document: 意外, 韓車, 排行榜, 聽說, 耗油
topic :1 Document: 特斯拉, 充電, 電動車, 南韓, 小時
topic :1 Document: 充電, 車款, 韓國, 北美, 零下
topic :5 Document: 萬元, 台幣, 人民幣, 特斯拉, 售價
topic :0 Document: 需求, 供給, 自然, 市場, 廠商
topic :1 Document: 充電, 國道, 服務區, 車位, 電動車
topic :5 Document: 降價, 大陸, 市場, 來到, 台灣
topic :5 Document: 來源, 按摩, 千輛, 僅有, 優勢
topic :5 Document: 銷量, 澳洲, 電動車, 中型, 地位
topic :1 Document: 車主, 特斯拉, 貼膜, 白色, 能夠
topic :3 Document: 電動車, 火災, 原文, 迅速, 火勢
topic :5 Document: 台灣, 特斯拉, 萬元, 降價, 售價
topic :0 Document: 汽車, 軟體, 定義, 未來, 遊戲
topic :4 Document: 公司, 含稅, 方便, 充電, 現在
topic :4 Document: 車頭, 牌子, 鼻孔, 不

In [64]:
car.head(20)

,artDate,artTitle,artContent,content,topic_label,words
0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,5,"[八卦, 官網, 特斯拉, 二手車, 第三方, 平台, 便宜, 黨會, 勝利]"
1,2023-01-01 22:42:25,問題特斯拉久了會掉電嗎,各位前輩好想買中古特斯拉里程約萬上下的擔心電池久了會掉電越充越少電里程數越少開了年後車子變成...,各位前輩好想買中古特斯拉里程約萬上下的擔心電池久了會掉電越充越少電里程數越少開了年後車子變成...,1,"[前輩, 中古, 特斯拉, 里程, 擔心, 電池, 掉電, 越充, 越少電, 里程, 數越,..."
2,2023-01-02 12:47:31,新聞開特斯拉滑手機被勸嗆網友有毛,原文連結原文內容記者王家珩徐兆緯台北報導前電競世界冠軍日前開直播雖然用自動駕駛但雙手卻完全離...,原文連結原文內容記者王家珩徐兆緯台北報導前電競世界冠軍日前開直播雖然用自動駕駛但雙手卻完全離...,3,"[原文, 連結, 原文, 內容, 王家, 徐兆緯, 台北, 前電, 世界冠軍, 日前, 直播..."
3,2023-01-02 18:01:04,討論是不是真的有些四輪覺得自己是大爺,如題本身開特斯拉最近出門覺得找車位很煩都騎家裡超舊機車基本上在路上有機車道騎機車道沒機車道就...,如題本身開特斯拉最近出門覺得找車位很煩都騎家裡超舊機車基本上在路上有機車道騎機車道沒機車道就...,4,"[特斯拉, 最近, 出門, 車位, 很煩, 騎家裡, 超舊, 機車, 路上, 有機, 車道,..."
4,2023-01-02 19:21:24,新聞美國汽車召回排名揭曉第一名超過,原文連結原文內容美國汽車召回排名揭曉第一名超過萬輛車進廠處理結束迎來許多年的數據也立刻揭曉來...,原文連結原文內容美國汽車召回排名揭曉第一名超過萬輛車進廠處理結束迎來許多年的數據也立刻揭曉來...,3,"[原文, 連結, 原文, 內容, 美國, 汽車, 召回, 排名, 揭曉, 第一名, 超過, ..."
5,2023-01-02 22:03:33,新聞美國汽車召回排名揭曉第一名超過,,,0,[]
6,2023-01-03 10:14:48,新聞美國汽車召回排名揭曉第一名超過,先上美國原文新聞原圖這篇新聞報導的時候特斯拉當年只有個召回特斯拉的個召回裡面有個都只是更新修...,先上美國原文新聞原圖這篇新聞報導的時候特斯拉當年只有個召回特斯拉的個召回裡面有個都只是更新修...,3,"[美國, 原文, 新聞, 原圖, 這篇, 新聞報導, 特斯拉, 當年, 召回, 特斯拉, 召..."
7,2023-01-03 12:31:28,新聞又是破紀錄的一年特斯拉年交車,原文連結原文內容又是破紀錄的一年特斯拉年交車成績單揭曉全年總量突破萬輛大關特斯拉正式對外公布...,原文連結原文內容又是破紀錄的一年特斯拉年交車成績單揭曉全年總量突破萬輛大關特斯拉正式對外公布...,5,"[原文, 連結, 原文, 內容, 破紀錄, 一年, 特斯拉, 年交車, 成績單, 揭曉, 總..."
8,2023-01-03 12:32:48,情報年月份臺灣汽車市場銷售報告,月份台灣新車市場共掛牌輛電動車賣到第一名絕對是亮點先來看看進口和國產前名還有哪些車款進榜稍後...,月份台灣新車市場共掛牌輛電動車賣到第一名絕對是亮點先來看看進口和國產前名還有哪些車款進榜稍後...,5,"[月份, 台灣, 新車, 市場, 掛牌, 電動車, 第一名, 亮點, 進口, 國產, 前名,..."
9,2023-01-04 09:55:23,新聞人奇蹟生還特斯拉衝破公路圍欄墜,原文連結原文內容人奇蹟生還特斯拉衝破公路圍欄墜公尺懸崖成廢鐵聯合報編譯張君堯即時報導美國加州...,原文連結原文內容人奇蹟生還特斯拉衝破公路圍欄墜公尺懸崖成廢鐵聯合報編譯張君堯即時報導美國加州...,3,"[原文, 連結, 原文, 內容, 奇蹟, 生還, 特斯拉, 衝破, 公路, 圍欄, 公尺, ..."


## 使用者體驗面深入分析

## 安全性方面深入分析

## 性價比方面深入分析

In [65]:
df = pd.read_csv("data/ppt_tesla_classified_cleaned.csv")
df_price = df[df["class"]== "性價比方面"]
df_price['sentence'] = df_price['sentence'].str.split("[,，。！!？?]{1,}")
# tmp = df_price.copy()
# drop_cols = ['system_id', 'sentence','class','LLM']
# tmp.drop(drop_cols, axis = 1, inplace = True)
# tmp.to_csv("self_data/tmmm.csv")
sent_df = df_price.explode('sentence').reset_index(drop=True)

sent_df['sentence'] = sent_df['sentence'].str.replace(r'[^\w\s]+', '', regex=True).astype(str)
sent_df = sent_df[sent_df["sentence"].str.len() > 2]
sent_df
sent_df

,system_id,artDate,artTitle,artContent,sentence,LLM,sentiment,class
0,0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面
1,1,2023-01-01 22:42:25,問題特斯拉久了會掉電嗎,各位前輩好想買中古特斯拉里程約萬上下的擔心電池久了會掉電越充越少電里程數越少開了年後車子變成...,各位前輩好想買中古特斯拉里程約萬上下的擔心電池久了會掉電越充越少電里程數越少開了年後車子變成...,"{""文章分類"":""性價比方面"",""情緒"":""負面"",}",負面,性價比方面
2,4,2023-01-02 19:21:24,新聞美國汽車召回排名揭曉第一名超過,原文連結原文內容美國汽車召回排名揭曉第一名超過萬輛車進廠處理結束迎來許多年的數據也立刻揭曉來...,原文連結原文內容美國汽車召回排名揭曉第一名超過萬輛車進廠處理結束迎來許多年的數據也立刻揭曉來...,"{""文章分類"": ""性價比方面"",""情緒"": ""負面"",}",負面,性價比方面
3,5,2023-01-02 22:03:33,新聞美國汽車召回排名揭曉第一名超過,NaN,nan,"{""文章分類"":""性價比方面"",""情緒"":""負面"",}",負面,性價比方面
4,7,2023-01-03 12:31:28,新聞又是破紀錄的一年特斯拉年交車,原文連結原文內容又是破紀錄的一年特斯拉年交車成績單揭曉全年總量突破萬輛大關特斯拉正式對外公布...,原文連結原文內容又是破紀錄的一年特斯拉年交車成績單揭曉全年總量突破萬輛大關特斯拉正式對外公布...,"{""文章分類"": ""性價比方面"",""情緒"": ""正面""}",正面,性價比方面
...,...,...,...,...,...,...,...,...
564,1574,2024-04-29 11:50:48,討論這感覺已經不對,之前特斯拉傳要推出平價車我本來要買看到這個消息想說等看看特斯拉出的平價車因為我對於內裝性能外...,之前特斯拉傳要推出平價車我本來要買看到這個消息想說等看看特斯拉出的平價車因為我對於內裝性能外...,"{""文章分類"": ""性價比方面"",""情緒"": ""積極"",}",積極,性價比方面
565,1579,2024-04-30 10:33:28,新聞特斯拉全新入門電動車準備量產,智借社團圖片一用看起來按照特斯拉的定價模式大概就是產地價乘上倍稅金所以萬美元到台灣應該不到萬...,智借社團圖片一用看起來按照特斯拉的定價模式大概就是產地價乘上倍稅金所以萬美元到台灣應該不到萬...,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面
566,1580,2024-04-30 10:33:35,新聞特斯拉全新入門電動車準備量產,哪有到倍那種是油車電動車現在沒有貨物稅以美國現在價格簡單計算台灣萬只比美國售價直接換算貴一點...,哪有到倍那種是油車電動車現在沒有貨物稅以美國現在價格簡單計算台灣萬只比美國售價直接換算貴一點...,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面
567,1583,2024-04-30 19:19:58,問題,最近物色下一台車原本抓左右上週去看了特斯拉這周去看了爆預算爆很大目前用車都是十年以上換車里程...,最近物色下一台車原本抓左右上週去看了特斯拉這周去看了爆預算爆很大目前用車都是十年以上換車里程...,"{""文章分類"": ""性價比方面"",""情緒"": ""負面"",}",負面,性價比方面


In [66]:
jieba.set_dictionary("dict/dict.txt")
word_df = sent_df.assign(word = sent_df['sentence'].apply(jieba.lcut)).explode('word').drop(['sentence'], axis=1)


word_df = word_df.loc[word_df['word'].str.len() > 1]

word_df

Building prefix dict from /Users/arron/Desktop/SMA_project/SMA_project/dict/dict.txt ...
2024-06-05 23:28:41,815 : DEBUG : Building prefix dict from /Users/arron/Desktop/SMA_project/SMA_project/dict/dict.txt ...
Loading model from cache /var/folders/s0/vxcd6261437fdmx2j76q9tj00000gn/T/jieba.ub420de3365ed380a966939938bdca42e.cache
2024-06-05 23:28:41,816 : DEBUG : Loading model from cache /var/folders/s0/vxcd6261437fdmx2j76q9tj00000gn/T/jieba.ub420de3365ed380a966939938bdca42e.cache
Loading model cost 0.228 seconds.
2024-06-05 23:28:42,044 : DEBUG : Loading model cost 0.228 seconds.
Prefix dict has been built successfully.
2024-06-05 23:28:42,045 : DEBUG : Prefix dict has been built successfully.


,system_id,artDate,artTitle,artContent,LLM,sentiment,class,word
0,0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面,八卦
0,0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面,官網
0,0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面,特斯拉
0,0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面,二手車
0,0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面,很多
...,...,...,...,...,...,...,...,...
568,1584,2024-04-30 19:53:28,問題,吧你家裡沒有充電樁工作地點也沒有每年里程數不夠高沒必要買不過你預算到有喜歡飆車的話那不要買我...,"{""文章分類"": ""性價比方面"",""情緒"": ""負面"",}",負面,性價比方面,喜歡
568,1584,2024-04-30 19:53:28,問題,吧你家裡沒有充電樁工作地點也沒有每年里程數不夠高沒必要買不過你預算到有喜歡飆車的話那不要買我...,"{""文章分類"": ""性價比方面"",""情緒"": ""負面"",}",負面,性價比方面,飆車
568,1584,2024-04-30 19:53:28,問題,吧你家裡沒有充電樁工作地點也沒有每年里程數不夠高沒必要買不過你預算到有喜歡飆車的話那不要買我...,"{""文章分類"": ""性價比方面"",""情緒"": ""負面"",}",負面,性價比方面,不要
568,1584,2024-04-30 19:53:28,問題,吧你家裡沒有充電樁工作地點也沒有每年里程數不夠高沒必要買不過你預算到有喜歡飆車的話那不要買我...,"{""文章分類"": ""性價比方面"",""情緒"": ""負面"",}",負面,性價比方面,覺得


In [67]:
stopwords = [line.strip() for line in open('dict/stopwords.txt', 'r', encoding='utf-8').readlines()] 
after_clear_df = word_df[~word_df['word'].isin(stopwords)]
after_clear_df

,system_id,artDate,artTitle,artContent,LLM,sentiment,class,word
0,0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面,八卦
0,0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面,官網
0,0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面,特斯拉
0,0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面,二手車
0,0,2023-01-01 10:48:19,情報慟特斯拉二手價跌好多,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,"{""文章分類"": ""性價比方面"",""情緒"": ""正面"",}",正面,性價比方面,三方
...,...,...,...,...,...,...,...,...
568,1584,2024-04-30 19:53:28,問題,吧你家裡沒有充電樁工作地點也沒有每年里程數不夠高沒必要買不過你預算到有喜歡飆車的話那不要買我...,"{""文章分類"": ""性價比方面"",""情緒"": ""負面"",}",負面,性價比方面,不夠
568,1584,2024-04-30 19:53:28,問題,吧你家裡沒有充電樁工作地點也沒有每年里程數不夠高沒必要買不過你預算到有喜歡飆車的話那不要買我...,"{""文章分類"": ""性價比方面"",""情緒"": ""負面"",}",負面,性價比方面,必要
568,1584,2024-04-30 19:53:28,問題,吧你家裡沒有充電樁工作地點也沒有每年里程數不夠高沒必要買不過你預算到有喜歡飆車的話那不要買我...,"{""文章分類"": ""性價比方面"",""情緒"": ""負面"",}",負面,性價比方面,預算
568,1584,2024-04-30 19:53:28,問題,吧你家裡沒有充電樁工作地點也沒有每年里程數不夠高沒必要買不過你預算到有喜歡飆車的話那不要買我...,"{""文章分類"": ""性價比方面"",""情緒"": ""負面"",}",負面,性價比方面,喜歡


In [68]:
freq_df_clear = pd.DataFrame(pd.value_counts(after_clear_df['word'])).sort_values(by = ['count'], ascending = False).reset_index()
freq_df_clear

,word,count
0,特斯拉,1148
1,電動車,1048
2,汽車,732
3,中國,646
4,電池,594
...,...,...
11575,新三,1
11576,妻子,1
11577,僅占,1
11578,其子,1


In [69]:
freq_dict_nostop = freq_df_clear.set_index('word').to_dict()
freq_dict_nostop = freq_dict_nostop['count']
freq_dict_nostop

{'特斯拉': 1148,
 '電動車': 1048,
 '汽車': 732,
 '中國': 646,
 '電池': 594,
 '台灣': 457,
 '市場': 434,
 '品牌': 415,
 '充電': 403,
 '原文': 310,
 '美國': 287,
 '價格': 274,
 '電車': 256,
 '公司': 249,
 '目前': 238,
 '降價': 224,
 '銷量': 223,
 '銷售': 220,
 '成本': 203,
 '心得': 200,
 '車主': 199,
 '現在': 199,
 '問題': 185,
 '生產': 184,
 '萬元': 179,
 '連結': 173,
 '全球': 171,
 '豐田': 170,
 '車廠': 165,
 '亞迪': 164,
 '油車': 160,
 '歐洲': 150,
 '日本': 148,
 '售價': 146,
 '內容': 142,
 '說明': 142,
 '消費者': 138,
 '公里': 138,
 '便宜': 134,
 '萬輛': 130,
 '車款': 128,
 '知道': 127,
 '使用': 126,
 '美元': 126,
 '技術': 122,
 '車型': 121,
 '時間': 119,
 '福特': 119,
 '電動': 118,
 '超過': 110,
 '成長': 107,
 '新聞': 103,
 '製造商': 103,
 '看到': 101,
 '車輛': 101,
 '產品': 100,
 '需要': 99,
 '動力': 99,
 '傳統': 97,
 '保養': 97,
 '未來': 95,
 '能源': 94,
 '車子': 94,
 '德國': 92,
 '車價': 92,
 '數據': 92,
 '里程': 91,
 '產業': 91,
 '直接': 88,
 '台幣': 87,
 '認為': 87,
 '工廠': 85,
 '競爭': 85,
 '續航': 83,
 '推出': 83,
 '系統': 83,
 '感覺': 82,
 '需求': 82,
 '一下': 82,
 '大陸': 80,
 '二手': 79,
 '車商': 78,
 '一台': 78,
 '台車': 77,
 '費用': 77,
 '維

In [72]:
FontPath = './SourceHanSansTW-Regular.otf' # 設定字型
wordcloud_nostop = WordCloud(background_color = 'white', width = 800, height = 400, font_path = FontPath, max_words = 100)
wordcloud_nostop.generate_from_frequencies(freq_dict_nostop)

plt.figure(figsize = (15,12))
plt.imshow(wordcloud_nostop)
plt.axis('off')
plt.show()

OSError: cannot open resource

## 服務方面深入分析